In [5]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/news/fake-news/fake-news-negatives-summarization.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/news/fake-news/fake-news-positives-summarization.json
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/news/fake-news/compressed-fake-news.zip

In [6]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/news/fake-news/summarization-augmentation/250-news-with-valid-hoax-label-summaries.json
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/news/fake-news/summarization-augmentation/600-news-with-valid-hoax-label-summaries.json
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/news/fake-news/summarization-augmentation/facts-summaries.json
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/news/fake-news/summarization-augmentation/hoax-summaries.json
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/news/fake-news/summarization-augmentation/malaysia-scraping-syazanihussin-summaries.json

In [7]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/news/fake-news/indonesian/hoax.csv
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/news/fake-news/indonesian/facts.csv
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/news/fake-news/indonesian/250%20news%20with%20valid%20hoax%20label.csv
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/news/fake-news/indonesian/600%20news%20with%20valid%20hoax%20label.csv
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/news/fake-news/malaysia-scraping-syazanihussin.csv

In [8]:
import re
from unidecode import unidecode

def cleaning(string):
    string = unidecode(string)
    string = string.replace('SPPPPLIIIT>', ' ').replace('SPPPPLIIIT', ' ')
    string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', string)
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    return ' '.join(string)

In [9]:
import pandas as pd

df = pd.read_csv('250 news with valid hoax label.csv', sep = ';',encoding = "ISO-8859-1")

In [10]:
X, Y = [], []

for i in range(len(df)):
    X.append(df.iloc[i,0])
    Y.append(df.iloc[i,1])

In [11]:
df = pd.read_csv('600 news with valid hoax label.csv', sep = ';',encoding = "ISO-8859-1")

for i in range(len(df)):
    X.append(df.iloc[i,0])
    Y.append(df.iloc[i,1])

In [12]:
import json

with open('250-news-with-valid-hoax-label-summaries.json') as fopen:
    data = json.load(fopen)

In [13]:
for i in data:
    X.extend(i[0])
    Y.extend([i[1]] * len(i[0]))

In [14]:
with open('600-news-with-valid-hoax-label-summaries.json') as fopen:
    data = json.load(fopen)
    
for i in data:
    X.extend(i[0])
    Y.extend([i[1]] * len(i[0]))

In [15]:
df = pd.read_csv('facts.csv', sep = ',',encoding = "ISO-8859-1")
for i in range(len(df)):
    X.append(df.iloc[i,1])
    Y.append(df.iloc[i,2])

In [16]:
df = pd.read_csv('hoax.csv', sep = ',',encoding = "ISO-8859-1")
for i in range(len(df)):
    X.append(df.iloc[i,1])
    Y.append(df.iloc[i,2])

In [17]:
with open('hoax-summaries.json') as fopen:
    data = json.load(fopen)
    
for i in data:
    X.extend(i[0])
    Y.extend([i[1]] * len(i[0]))

In [18]:
with open('facts-summaries.json') as fopen:
    data = json.load(fopen)
    
for i in data:
    X.extend(i[0])
    Y.extend([i[1]] * len(i[0]))

In [19]:
df = pd.read_csv('malaysia-scraping-syazanihussin.csv')
for i in range(len(df)):
    X.append(df.iloc[i,0])
    Y.append(df.iloc[i,1])

In [20]:
with open('malaysia-scraping-syazanihussin-summaries.json') as fopen:
    data = json.load(fopen)
    
for i in data:
    X.extend(i[0])
    Y.extend([i[1]] * len(i[0]))

In [21]:
# !unzip compressed-fake-news.zip

In [22]:
from glob import glob

X_, Y_ = [], []

for file in glob('positive/*.json'):
    with open(file) as fopen:
        data = json.load(fopen)
    X_.extend(data)
    Y_.extend(['fake'] * len(data))

In [23]:
for file in glob('negative/*.json'):
    with open(file) as fopen:
        data = json.load(fopen)
    X_.extend(data)
    Y_.extend(['real'] * len(data))

In [24]:
with open('fake-news-negatives-summarization.json') as fopen:
    data = json.load(fopen)
    
for i in data:
    X_.extend(i)
    Y_.extend(['real'] * len(i))

In [25]:
with open('fake-news-positives-summarization.json') as fopen:
    data = json.load(fopen)
    
for i in data:
    X_.extend(i)
    Y_.extend(['fake'] * len(i))

In [26]:
from tqdm import tqdm

selected_X, selected_Y = [], []
mapping_label = {'fake': 0, 'hoax': 0, 'real': 1, 'valid': 1, 'fact': 1}

for i in tqdm(range(len(X_))):
    try:
        t = cleaning(X_[i])
        if len(t) > 100:
            selected_X.append(t)
            selected_Y.append(mapping_label[Y_[i].lower()])
    except:
        pass
        
len(selected_X), len(selected_Y)

100%|██████████| 59935/59935 [00:21<00:00, 2777.64it/s]


(50284, 50284)

In [28]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(selected_X, selected_Y, test_size = 0.1)
len(train_X), len(test_X)

(45255, 5029)

In [29]:
train_X = train_X + selected_X
train_Y = train_Y + selected_Y

In [30]:
from sklearn.utils import shuffle

train_X, train_Y = shuffle(train_X, train_Y)

In [31]:
from malaya.text.bpe import WordPieceTokenizer

tokenizer = WordPieceTokenizer('BERT.wordpiece', do_lower_case = False)
# tokenizer.tokenize('halo nama sayacomel')

In [32]:
def get_tokens(X, Y, maxlen = 1024):

    input_ids, input_masks = [], []
    actual_l = []

    for i in tqdm(range(len(X))):
        text = X[i]
        tokens_a = tokenizer.tokenize(text)
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_id)

        if len(input_id) <= maxlen:
            input_ids.append(input_id)
            input_masks.append(input_mask)
            actual_l.append(Y[i])
            
    return input_ids, actual_l

In [33]:
train_input_ids, train_actual_l = get_tokens(train_X, train_Y)

100%|██████████| 95539/95539 [15:20<00:00, 103.74it/s]


In [34]:
test_input_ids, test_actual_l = get_tokens(test_X, test_Y)

100%|██████████| 5029/5029 [00:47<00:00, 104.84it/s]


In [35]:
import pickle

with open('relevancy-fastformer.pkl', 'wb') as fopen:
    pickle.dump([train_input_ids, train_actual_l, test_input_ids, test_actual_l], fopen)